<a href="https://colab.research.google.com/github/RCortez25/Scientific-Machine-Learning/blob/main/Differential_equations/SIR(NODE)_autonomous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

# Code walkthrough

In [ ]:
# A
using ComponentArrays, Lux, DiffEqFlux, OrdinaryDiffEq, Optimization, OptimizationOptimJL,
    OptimizationOptimisers, Random, Plots, ModelingToolkit

# B
random_number_generator = Random.default_rng()
Random.seed!(rng, 42)

# C
#------ Generate ground-truth data
# Autonomous system where t is the independent variable
@parameters β γ N
@independent_variables t
@variables S(t) I(t) R(t)
Dt = Differential(t)

eqs = [
    Dt(S) ~ -(β*S*I)/N,
    Dt(I) ~ ((β*S*I)/N) - γ*I,
    Dt(R) ~ γ*I
]

@named system = ODESystem(eqs, t, [S, I, R], [β, γ, N])
simplified = structural_simplify(system)

parameter_map = Dict(β => 0.3, γ => 0.1, N => 1000)
initial_conditions = Dict(I => 1, R => 0, S => 1000 - 1 - 0)
timespan = (0.0, 160.0) # in days

problem = ODEProblem(simplified,
                     merge(initial_conditions, parameter_map),
                     timespan)

solution = solve(problem, Tsit5(), saveat=1)
ground_truth = Array(solution)

**A** - Importing the packages
*   `ComponentArrays` is for packaging parameters into a single vector with names and structure
*   `DiffEqFlux` for using `NeuralODE`
*   `OptimizationOptimisers` adapter for `Optimisers.jl`, for using ADAM.
*   `Random` for seeding random number generators for reproducibility

**B** - Create a seeded random number generator for reproducibility

**C** - The rest of the code is used for solving the ODE system and generate ground-truth data for comparing with the NN results. `ground_truth` stores the results of the integrator. Note that `saveat=1` means that we're saving for each day, 1 day at a time.

In [ ]:
# A
const input_dimension = 3 # S, I, R
const output_dimension = 3 # Derivatives of S, I, R

# B
layer_0 = Lux.Dense(input_dimension, 32, Lux.tanh)
layer_1 = Lux.Dense(32, 32, Lux.tanh)
layer_2 = Lux.Dense(32, output_dimension)

# C
NN = Lux.Chain(
    layer_0,
    layer_1,
    layer_2
)

# D
NN64 = Lux.f64(NN)

# E
parameters, state = Lux.setup(random_number_generator, NN64)

**A** - Defining the input and output dimensions. `const` locks them up in order to avoid accidental re-writing of the dimensions. In the case of NeuralODEs, as one is predicting the derivative, the inputs are $t,S,I,R$ in that order, and the output numbers represent the value of their derivatives $\dot{S},\dot{I},\dot{R}$.

**B** - Definition of the NN architecture, in this case, 3 layers:
*   `layer_0` is the input layer, recieves 4 inputs and outputs 32 numbers (this is arbitrary and can be changed) using `tanh` activation function.
*   `layer_1` First hidden layer with 32 inputs (from the previous layer) and 32 outputs using `tanh` activation layer.
*   `layer_2` is the output layer. It recieves 32 inputs (from the previous layer) and outputs 3 numbers, namely, the derivatives as stated before.

**C** - Creating of the NN using the defined layers

**D** - Makes the NN use `Float64` for better performance with the solvers

**E** - Initializing the network. It returns two objects:
*   `parameters` which is the set of all trainable parameters and biases to be optimized later during training.
*   `state` all non-trainable internal states some layers keep (BatchNorm running means, etc).

In [ ]:
# A
neural_ODE_problem = NeuralODE(NN64, timespan, Tsit5(); saveat = 1)

# B
function neural_ode_predictions(parameters, state)
    u0 = Float64[
        initial_conditions[S],
        initial_conditions[I],
        initial_conditions[R],
    ]
    trajectory, new_state = neural_ODE_problem(u0, parameters, state)
    return Array(trajectory)
end

**A** - Builds the solver layer. It prepares everything so that when called, an ODE whose RHS is the neural network passed to it, is integrated. It

*   Indicates that the ODE's RHS (the ODE rule) is the neural network passed to it.
*   Stores how to solve it: the time span, the algorithm, at what time steps to solve it, and tolerances like `reltol`, `abstol`, not used in this example.

This is just the constructor, where everything is configured, but not run yet. The returned object is called as `neural_ODE_problem(initial_conditions, parameters, state)`.

**B** - Function for a NeuralODE forward pass. It accepts the trainable parameters of the NN and the state (BatchNorms, etc., in this case, the system is stateless, that is, just Dense + tanh).

*   `u0` is the array of initial conditions. Earlier, we defined these using a dictionary, but we need to unpack them into a vector.
*   The `NeuralODE` is run, passing it the initial conditions, trainable parameters and the state. This integrates the ODE with the NN as the RHS using the solver, time span, etc, as defined above. This returns two objects:
    *   `trajectory`: The trajectory matrix, in this case, of shape roughly `(3,161)`, that is, 3 rows (for each S, I, and R), and 161 columns for each day the simulation was instructed to run (`saveat=1.0`).
    * `new_state`: The updated state (BatchNorms, etc), but in this case, since the system is stateless, there's no need for the use of this variable.

Changing `parameters` amounts to changing the NN weights, and this changes the predicted values and the integrated trajectory.

In the end, the function returns the `trajectory` as a Julia `Array`, ignoring `state` in this particular case.

In [ ]:
# A
function loss_neuralode(parameters, state)
    predicted_trajectory = neural_ode_predictions(parameters, state)
    loss = sum(abs2, ground_truth .- predicted_trajectory)
    return loss, predicted_trajectory
end

# Notes

1.  The `NeuralODE` constructor expects a function as one of its parameters. Specifically, a function of the form

$$f(u,p,t)⟶du/dt$$

(though one often only writes $f(u,p,t)⟶du$) because the solver, who lives inside `NeuralODE` (for example, `Tsit5()`), integrates that $du/dt$ and gives $u=[S,I,R]$. That is, the output of `NeuralODE` is the state `u` that can be called with initial conditions in order to obtain a whole trajectory.